### Housekeeping - Setup & wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using _asadm_.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Running _asadm_ in iJava
We can run _asadm_ commands inline. Below, we will use the truncation command, which normally requires an interactive confirmation, which we will skip by using the _--no-warn_ flag. No output will be displayed.


In [2]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"

#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [4]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>7.2.1</version>
  </dependency>
</dependencies>

####  Java Client Imports and Test code

In [5]:
//Required Imports
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
import com.aerospike.client.Operation;
import com.aerospike.client.exp.Exp;
import com.aerospike.client.exp.ExpOperation;
import com.aerospike.client.exp.ExpWriteFlags;
import com.aerospike.client.exp.Expression;
import com.aerospike.client.cdt.ListOperation;
import com.aerospike.client.exp.ListExp;

import com.aerospike.client.AerospikeException;
import com.aerospike.client.ResultCode;

AerospikeClient client = new AerospikeClient("127.0.0.1", 3000);
Key key = new Key("test", "testset", "key_boolean");
WritePolicy wPolicy = new WritePolicy();

Value.UseBoolBin = true;

Bin b0 = new Bin("iBin", Value.get(1)); //int bin
Bin b1 = new Bin("foo", Value.get(true)); //bool bin
// By default, Value.UseBoolBin = false;
// In that case, this creates b1 as an integer bin instead of a boolean bin. 

client.delete(wPolicy,key);
client.put(wPolicy, key, b0, b1);
/*
client.operate(wPolicy, key,
               Operation.put(b0),
               ExpOperation.write("foo", Exp.build(Exp.val(true)), ExpWriteFlags.DEFAULT)
               );
*/
System.out.println("Record: "+ client.get(null, key)); //check record got inserted

Expression newBoolExp = Exp.build(                        
                          Exp.cond( 
                              Exp.or(Exp.boolBin("foo"), 
                                     Exp.gt(Exp.intBin("iBin"), Exp.val(300))
                                    ),
                              Exp.val(true), 
                              Exp.val(false)
                          )
                        );

client.operate( wPolicy, key,   
          ExpOperation.write("result", newBoolExp, ExpWriteFlags.DEFAULT) 
         );
         
System.out.println("Record: " + client.get(null, key));



Record: (gen:1),(exp:450844489),(bins:(iBin:1),(foo:true))
Record: (gen:2),(exp:450844490),(bins:(iBin:1),(foo:true),(result:true))
